In [ ]:
%pip -q install google-genai

In [31]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [ ]:
# Importação dos ADKs
%pip install google-adk
from IPython import get_ipython
from IPython.display import display


In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [ ]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [ ]:
# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview

In [ ]:
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-1.5-flash"
MODEL_ID = "gemini-2.0-flash"


In [ ]:
# objetivo da célula: testar o modelo Gemini diretamente
from google.generativeai import GenerativeModel

# (Não é necessário rodar sempre) Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
# Use GenerativeModel diretamente após a nova importação
model = GenerativeModel("gemini-1.5-flash")
resposta = model.generate_content("Quando é a próxima Imersão IA com Google Gemini da Alura?")
display(Markdown(f"**Resposta:**\n\n{resposta.text}"))

**Resposta:**

Não tenho acesso a informações em tempo real, incluindo calendários de eventos de empresas como a Alura.  Para saber a data da próxima Imersão IA com Google Gemini da Alura, recomendo que você:

* **Visite diretamente o site da Alura:** Procure na seção de cursos ou eventos.  Geralmente, as datas das próximas imersões são anunciadas lá.
* **Verifique as redes sociais da Alura:**  Busque em seus perfis no Facebook, Instagram, LinkedIn, etc., por anúncios sobre a imersão.
* **Assine a newsletter da Alura:**  Eles frequentemente anunciam novos cursos e imersões por e-mail.


Lembre-se que a disponibilidade e as datas das imersões podem variar.


In [ ]:
# (Não é necessário rodar ). O objetivo da Célula é testar o modelo Gemini com a busca ativada Pergunta ao Gemini uma informação utilizando a busca do Google como contexto
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições foram até o dia 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini', 'próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: tecmundo.com.br, alura.com.br, youtube.com



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")


In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

# Editada: Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

import asyncio  # Importar para usar asyncio.sleep
from google.api_core import exceptions  # Importar para capturar exceções da API
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types  # Para criar conteúdos (Content e Part)


async def call_agent(agent: Agent, message_text: str) -> str:
    # Número máximo de tentativas
    max_retries = 5
    # Atraso inicial em segundos
    initial_delay = 1

    for attempt in range(max_retries):
        try:
            # Cria um serviço de sessão em memória (moved inside the function to ensure a new service per call if needed, or you can keep it outside if appropriate for your use case)
            # If session_service is intended to be shared, move this line back outside the function.
            session_service = InMemorySessionService()
            # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
            session = await session_service.create_session(app_name=agent.name, user_id="user1")
            # Cria um Runner para o agente
            runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
            # Cria o conteúdo da mensagem de entrada
            content = types.Content(role="user", parts=[types.Part(text=message_text)])

            final_response = ""
            # Itera assincronamente pelos eventos retornados durante a execução do agente
            async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
                if event.is_final_response():
                  for part in event.content.parts:
                    if part.text is not None:
                      final_response += part.text
                      final_response += "\n"
            # Se chegou aqui sem erro, retorna a resposta e sai do loop de retry
            return final_response

        except exceptions.ServerError as e:
            # Verifica se é um erro 503 UNAVAILABLE
            if e.code == 503:
                print(f"Erro 503 UNAVAILABLE na tentativa {attempt + 1}/{max_retries}. Tentando novamente...")
                # Calcula o atraso com exponencial backoff
                delay = initial_delay * (2 ** attempt)
                await asyncio.sleep(delay)
            else:
                # Se for outro ServerError, relança a exceção
                raise e
        except Exception as e:
            # Captura outros erros inesperados e relança
            print(f"Erro inesperado na tentativa {attempt + 1}/{max_retries}: {e}")
            raise e

    # Se todas as tentativas falharam, levanta uma exceção
    raise Exception(f"Falha após {max_retries} tentativas de chamar o agente {agent.name}.")

# Garanta que o resto do seu código que usa call_agent permaneça o mesmo,
# especialmente as partes que usam `await call_agent(...)`.

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos


In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model=MODELO_ROBUSTO,
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model=MODELO_RAPIDO,
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model=MODELO_RAPIDO,
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")


In [32]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    # Chame agente_buscador e AGUARDE seu resultado
    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    # Passe o resultado para a função de formatação
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    # Chame agente_planejador e AGUARDE seu resultado
    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    # Passe o resultado para a função de formatação
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    # Chame agente_redator e AGUARDE seu resultado
    # A variável 'post_redigido' é definida aqui com o resultado da função agente_redator
    post_redigido = await agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    # Passe o resultado para a função de formatação
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")

    # Chame agente_revisor e AGUARDE seu resultado
    # Aqui você usa a variável 'post_redigido' que já foi definida e contém o rascunho do post
    post_final = await agente_revisor(topico, post_redigido)
    print("\n--- 📑 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA

Maravilha! Vamos então criar o post sobre novidades em Agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais recentes e relevantes sobre Agentes de IA, vou usar o Google para procurar notícias e anúncios recentes sobre o tema, com foco nos últimos 30 dias. As pesquisas serão feitas em português para garantir que os resultados sejam relevantes para o contexto solicitado.
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e desenvolvimentos relevantes sobre Agentes de IA nos últimos meses:
> 
> 1.  **Adoção em Massa e Investimentos:** Em março de 2025, foi noticiado que os agentes de IA estão se consolidando como protagonistas na transformação digital, com investimentos globais bilionários e o Brasil acompanhando essa tendência através do Plano Brasileiro de Inteligência Artificial. Bancos e empresas estatais no Brasil já investiram mais de R$ 2 bilhões em projetos de IA.
> 
> 2.  **Aplicações Práticas em Negócios:** Em maio de 2025, vários artigos destacaram o aumento da demanda por agentes de IA para automação de negócios. Setores como e-commerce, atendimento ao cliente, marketing e finanças estão incorporando agentes de IA para analisar dados e sugerir soluções em tempo real.
> 
> 3.  **B3 Adota Agentes de IA:** Em abril de 2025, a B3 (a bolsa de valores brasileira) anunciou a massificação do uso de agentes de IA em seus produtos e serviços para transformar a vida dos clientes e facilitar o trabalho de seus colaboradores.
> 
> 4.  **Novos Agentes de Codificação do Google (Jules):** Durante o Google I/O 2025, foi apresentado o Jules, um novo agente de codificação de IA do Google em formato beta. Ele automatiza tarefas de programação, como correção de bugs e criação de novos recursos, integrando-se com o GitHub.
> 
> 5.  **Agentes de IA na Checkmob:** Em maio de 2025, a Checkmob anunciou o lançamento de agentes de IA (Zano e Thom) para facilitar a gestão de seus clientes, com foco na automação de tarefas operacionais e apoio à tomada de decisões.
> 
> 6.  **Microsoft Build 2025:** A Microsoft revelou sua visão para a era dos agentes de IA e a construção de uma web agentic aberta, com inovações como GitHub Copilot evoluído, Azure AI Foundry e Microsoft Entra Agent ID.
> 
> 7.  **Ferramentas para Criar Agentes de IA:** Plataformas como Botpress, LangChain, LlamaIndex, CrewAI, Dify e Synthflow estão se destacando como ferramentas para criar sistemas orientados por agentes de IA, facilitando a automação de interações e fluxos de trabalho.
> 
> Esses lançamentos e notícias indicam um crescimento significativo no desenvolvimento e adoção de agentes de IA em diversos setores, com um foco em automação, eficiência e novas aplicações práticas.
> 


--------------------------------------------------------------
Erro inesperado na tentativa 1/5: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}]}}


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}]}}

In [35]:
# Função para orquestrar a execução do sistema de criação de posts
async def main():
    print("🚀 Iniciando o Systema de Criação de Posts 🚀\n")
    topico = input("Digite o TÓPICO sobre o qual você deseja criar um post: ")

    if not topico:
        print("Você não digitou um tópico.")
        return

    # Executa os agentes na sequência
    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    plano_de_post = await agente_planejador(topico, lancamentos_buscados)
    post_redigido = await agente_redator(topico, plano_de_post)
    post_final = await agente_revisor(topico, post_redigido)

    # Exibe o post final revisado
    print("\n--- Post Final ---\n")
    display(to_markdown(post_final))

# Chamando a função principal
await main()


🚀 Iniciando o Systema de Criação de Posts 🚀

Digite o TÓPICO sobre o qual você deseja criar um post: Agentes de IA
Erro inesperado na tentativa 1/5: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}]}}


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}]}}